# Proton Radiography: Creating Synthetic Proton Radiographs by Particle Tracing

Proton radiography is an diagnostic technique often used to interrogate the electric and magnetic fields inside high energy density plasmas. The area of interest is positioned between a bright source of protons and a detector plane. Electric and magnetc fields in the plasma deflect the protons, producing patterns on the detector. Since this represents a non-linear and line-integrated measurement of the fields, the interpretation of these "proton radiographs" is complicated.

The SimPrad module contains tools to create synthetic proton radiographs given a grid of electric and magnetic field from either calculations or simulations. After the geometry of the problem has been set up during initialization, a partile tracing algorithm is run, tracking the positions of up to several million protons as they pass through the fields region. After all particles have reached the detector plane, a synthetic proton radiograph can be made by creating a 2D histogram in that plane.

In [ ]:
import astropy.constants as const
import astropy.units as u
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from plasmapy.diagnostics import proton_radiography as prad

Several exampels of 3D grid of fields are included in the package. This example contains the radial electric field created by a Gaussian sphere of electric charge.

In [ ]:
grid, E, B = prad.test_fields(mode='electrostatic gaussian sphere')

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.view_init(30,30)

s = slice(None,None,20) # Plot only every 6th point
ax.quiver(grid[s,s,s,0].to(u.mm).value, grid[s,s,s,1].to(u.mm).value,  grid[s,s,s,2].to(u.mm).value,
           E[s,s,s,0].value, E[s,s,s,1].value, E[s,s,s,2].value, length=1e-10)

ax.set_xlabel("X (mm)")
ax.set_ylabel("Y (mm)")
ax.set_zlabel("Z (mm)")
ax.set_title("Gaussian Potential Electric Field")

Prior to running the particle tracing algorithm, the simulation instance must be instatiated by providing some information about the setup, including:
    - The location of the proton souce with respect to the origin of the field grid. Only point sources are supported. 
    
    - The location of the center of the detector plane. The vector from this point to the origin of the field grid defines the normal vector of the detector plane. The horizontal axis of the detector plane is defined to be orthoganal to the z-axis of the field grid.
    
    - The energy of the protons. Lower energy protons will undergo more severe deflections.
    
The source and detector coordinates are entered as a 3-tuple in one of three coordinate systems: cartesian (x,y,z), spherical (r, theta, phi) or cylindrical (r, theta, z). All values should be astropy Quantities with units of either length or angle (degrees or radians) as appropriate. Note that the vector from the source to the detector should pass through the origin to maximize the number of particles that pass through the simulated fields.

In [ ]:
source = (0*u.mm,-10*u.mm, 0*u.mm)
detector = (0*u.mm, 100*u.mm, 0*u.mm) 

sim = prad.SimPrad(grid, E, B, source, detector, 
                   proton_energy=14*u.MeV, geometry='cartesian')

In order to run the simulation, a number of simulated particles must be selected (nparticles). The simulation timestep is automatically (and adaptively) calculated based on the proton energy, grid resolution, and field amplitudes.

In [ ]:
sim.run(nparticles=2e5, max_theta = np.pi/6*u.rad)

At the end of the simulation run, all particles are allowed to coast until they reach the detector plane.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.view_init(30,120)
ax.set_xlabel('X (cm)')
ax.set_ylabel('Y (cm)')
ax.set_zlabel('Z (cm)')

# Plot the source-to-detector axis
ax.quiver(sim.source[0].to(u.cm).value, 
          sim.source[1].to(u.cm).value,
          sim.source[2].to(u.cm).value,
          sim.detector[0].to(u.cm).value, 
          sim.detector[1].to(u.cm).value,
          sim.detector[2].to(u.cm).value, color='black')   

# Plot the simulation field grid volume
ax.scatter(0,0,0, color='green', marker='s', linewidth=5, label='Grid')

# Plot the the proton source and detector plane locations
ax.scatter(sim.source[0].to(u.cm).value, 
           sim.source[1].to(u.cm).value,
           sim.source[2].to(u.cm).value, color='red', marker='*', linewidth=5, label='Source')

ax.scatter(sim.detector[0].to(u.cm).value, 
           sim.detector[1].to(u.cm).value,
           sim.detector[2].to(u.cm).value, color='blue', marker='*', linewidth=10, label='Detector')


# Plot the final proton positions of some (not all) of the protons
ind = slice(None,None,100)
ax.scatter(sim.r[ind,0].to(u.cm).value, sim.r[ind,1].to(u.cm).value, sim.r[ind,2].to(u.cm).value)

ax.legend()

A 'synthetic proton radiograph' can then be constructed by creating a 2D histograpm of proton positions in the image plane.

In [ ]:
size = np.array([[-20,20],[-20,20]])*1e-3*u.m
bins = [200,200]
hax, vax, intensity = sim.synthetic_radiograph(size=size, bins=bins)

fig, ax = plt.subplots()
plot = ax.pcolormesh(hax.to(u.cm).value, vax.to(u.cm).value,
                     intensity.T, cmap='Blues_r')

cb = fig.colorbar(plot)
cb.ax.set_ylabel("Intensity")

ax.set_aspect('equal')
ax.set_xlabel('X (cm), Image plane')
ax.set_ylabel('Z (cm), Image plane')
ax.set_title("Synthetic Proton Radiograph")

As expected, the outward-pointing electric field in the sphere has deflected the protons out of the central region, leaving a dark shadow.